In [4]:
# importing pandas and numpy 
import pandas as pd
import numpy as np
# importing data from csv file 
patient_df = pd.read_csv("D:\data science\Assignments\Basic stats - 2\Patient_Data.csv")      
billing_df = pd.read_csv("D:\data science\Assignments\Basic stats - 2\Billing_Data.csv")  

In [5]:
print("=== Patient Dataset Info ===")
print(patient_df.info())

print("\n=== Billing Dataset Info ===")
print(billing_df.info())


=== Patient Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None

=== Billing Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 by

In [33]:
billing_relevant = patient_df[['PatientID', 'Department', 'Doctor', 'BillAmount']]
print("\nBilling Relevant Columns:")
print(billing_relevant.head())



Billing Relevant Columns:
   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN


In [8]:
#Drop unnecessary administrative columns
patient_df.drop(['ReceptionistID', 'CheckInTime'], axis=1, inplace=True, errors='ignore')

In [9]:
# Select only billing-related columns
dept_bill = billing_relevant.groupby('Department')['BillAmount'].sum().reset_index()
dept_bill.rename(columns={'BillAmount': 'TotalBillAmount'}, inplace=True)
print("\nTotal Bill Amount per Department:")
print(dept_bill)


Total Bill Amount per Department:
    Department  TotalBillAmount
0   Cardiology          16200.0
1  Dermatology              0.0
2    Neurology              0.0
3  Orthopedics           7500.0


In [10]:
# Dropping duplicates
patient_df.drop_duplicates(subset='PatientID', keep='first', inplace=True)

In [34]:
#Fill missing BillAmount values
billing_relevant = patient_df[['PatientID', 'Department', 'Doctor', 'BillAmount']].copy()
mean_bill = billing_relevant['BillAmount'].mean()
billing_relevant.loc[:, 'BillAmount'] = billing_relevant['BillAmount'].fillna(mean_bill)

In [15]:
# Merged billing dataset with patient dataset on PatientID
merged_df = pd.merge(patient_df, billing_relevant, on='PatientID', how='left')

In [35]:
#Concatenate additional new patients for current week (row-wise)
new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Department': ['Cardiology', 'Orthopedics'],
    'Doctor': ['Dr. sagar', 'Dr. santosh'],
    'BillAmount': [5000, 3000]
})
merged_with_new = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)

In [36]:
# Concatenate new billing category columns (column-wise)
new_columns = pd.DataFrame({
    'InsuranceCovered': [True, False] * (len(merged_with_new) // 2) + [True] * (len(merged_with_new) % 2),
    'FinalAmount': merged_with_new['BillAmount'] * 0.9   
})


final_df = pd.concat([merged_with_new, new_columns], axis=1)

In [37]:
# final cleaned data set
print("\n=== Final Cleaned Dataset ===")
print(final_df.head())


=== Final Cleaned Dataset ===
   PatientID     Name Department_x   Doctor_x  BillAmount_x Department_y  \
0        101    Alice   Cardiology  Dr. Smith        5000.0   Cardiology   
1        102      Bob    Neurology   Dr. John           NaN    Neurology   
2        103  Charlie  Orthopedics    Dr. Lee        7500.0  Orthopedics   
3        104    David   Cardiology  Dr. Smith        6200.0   Cardiology   
4        105      Eva  Dermatology   Dr. Rose           NaN  Dermatology   

    Doctor_y  BillAmount_y Department Doctor  BillAmount  InsuranceCovered  \
0  Dr. Smith   5000.000000        NaN    NaN         NaN              True   
1   Dr. John   6233.333333        NaN    NaN         NaN             False   
2    Dr. Lee   7500.000000        NaN    NaN         NaN              True   
3  Dr. Smith   6200.000000        NaN    NaN         NaN             False   
4   Dr. Rose   6233.333333        NaN    NaN         NaN              True   

   FinalAmount  
0          NaN  
1        

In [38]:
# Department-wise revenue
dept_revenue = final_df.groupby('Department')['FinalAmount'].sum().reset_index()
print("\nDepartment-wise Revenue:")
print(dept_revenue)



Department-wise Revenue:
    Department  FinalAmount
0   Cardiology       4500.0
1  Orthopedics       2700.0


In [39]:
# Doctor performance (total billing)
doctor_performance = final_df.groupby('Doctor')['FinalAmount'].sum().reset_index()
print("\nDoctor-wise Performance:")
print(doctor_performance)


Doctor-wise Performance:
        Doctor  FinalAmount
0    Dr. sagar       4500.0
1  Dr. santosh       2700.0
